
# Week03 — Prompt Evaluation & Version Management (Langfuse + Notion + GitHub)

이 노트북은 **회사 회의 STT(회의록 텍스트) 1개**를 입력으로 받아,  
- **V0.0.1 (간단 요약 Prompt)**, **V0.0.2 (회사형 구조화 회의록 Prompt)** 두 번의 Prompt Engineering을 수행하고,  
- **Langfuse Tracing**을 남기며,  
- **Langfuse Datasets**에 평가용 샘플 2개를 업로드하고,  
- 이후 **Dataset Run/Evaluation**을 할 수 있도록 기본 코드를 제공합니다.

> ⚙️ 이 노트북은 `.env`에 저장된 다음 변수들을 사용합니다.
> - `LANGFUSE_PUBLIC_KEY`, `LANGFUSE_SECRET_KEY`, `LANGFUSE_HOST` (예: `https://cloud.langfuse.com` 또는 리전 URL)
> - `OPENAI_API_KEY` (또는 호환 LLM API 키)


In [8]:
!pip install langfuse


  Using cached langfuse-3.6.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached opentelemetry_exporter_otlp_proto_http-1.37.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached opentelemetry_exporter_otlp_proto_common-1.37.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached opentelemetry_proto-1.37.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached opentelemetry_sdk-1.37.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_api-1.37.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_semantic_conventions-0.58b0-py3-none-any.whl.metadata (2.4 kB)
Using cached langfuse-3.6.1-py3-none-any.whl (350 kB)
Using cached opentelemetry_exporter_otlp_proto_http-1.37.0-py3-none-any.whl (19 kB)
Using cached opentelemetry_exporter_otlp_proto_common-1.37.0-py3-none-any.whl (18 kB)
Using cached opentelemetry_proto-1.37.0-py3-none-any.whl (72 kB)
Using cached opentelemetry_sdk-1.37.0-py3-none-any.whl (131 kB)
Using cached opentelemetry_api-1.37.0-py3-none-any.whl (65 kB)
Using 

In [5]:
# --- 0) 설치 & 기본 설정 ---------------------------------------------------------
# 인터넷 환경에 따라 설치가 제한될 수 있습니다. 로컬 환경에서 실행하세요.
# %pip install -q langfuse python-dotenv openai requests

import os, json, time, uuid, sys
from pathlib import Path
from datetime import datetime
from typing import Dict, Any

# .env 로드
try:
    from dotenv import load_dotenv
    load_dotenv()
    print("Loaded .env")
except Exception as e:
    print("python-dotenv not available; make sure environment variables are set.")

LANGFUSE_PUBLIC_KEY = os.getenv("LANGFUSE_PUBLIC_KEY")
LANGFUSE_SECRET_KEY = os.getenv("LANGFUSE_SECRET_KEY")
LANGFUSE_HOST = os.getenv("LANGFUSE_HOST", "https://cloud.langfuse.com")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

print("Langfuse host:", LANGFUSE_HOST)
print("Langfuse public key set? ", bool(LANGFUSE_PUBLIC_KEY))
print("Langfuse secret key set? ", bool(LANGFUSE_SECRET_KEY))
print("OpenAI key set? ", bool(OPENAI_API_KEY))


Loaded .env
Langfuse host: https://cloud.langfuse.com
Langfuse public key set?  True
Langfuse secret key set?  True
OpenAI key set?  True


In [4]:

# --- 1) 회의 STT 텍스트 로드 ------------------------------------------------------
transcript_path = "LLMOPS 기반 서비스 제작 논의.txt"
# 노트북 실행 환경에 해당 파일이 없을 수 있으니 방어적으로 처리
transcript_text = ""
try:
    with open(transcript_path, "r", encoding="utf-8") as f:
        transcript_text = f.read()
        print("Loaded transcript:", transcript_path)
        print("Length:", len(transcript_text))
        print("\n--- Preview (first 800 chars) ---\n")
        print(transcript_text[:800])
except FileNotFoundError:
    print("Transcript file not found at:", transcript_path)
    transcript_text = "회의 안건: LLM 기반 평가 파이프라인 정리, 루브릭 및 데이터셋 설계 논의. 참석자: A,B,C... (샘플 텍스트)"


Loaded transcript: LLMOPS 기반 서비스 제작 논의.txt
Length: 32345

--- Preview (first 800 chars) ---

﻿LLMOPS 기반 서비스 제작 논의
2025.03.24 월 오후 2:06 ・ 71분 15초
차성재


참석자 1 00:11
그리고 스테이만 오케니안 그다음에 리즘 디테일 트랜지션 컨플루션 그래머 있고 문장 센텐스 컴플렉스티 플렉스티 있고 그리고 스타일 이렇게 맞죠?
스타일 맞나 원트 프리 프로 x 플리 마 텐 각각에 대해서 BDP가 있잖아요.
그리고 이게 이게 이제 총 4가지로 해가지고 그 베이스 인터미리엣 지금 어드벤스 엑스프리가 엑스프레티 이렇게 있는 거고 총 요 로보인 게 똑같은 게 지금 내 네 세트가 있어요.
그래서 이 부장님이 해 주시는 이제 여기서 이게 두 번째로 데이터 생성인데 이 부분이 오토메이션이 옵티메이션까지 안 간다 하더라도 지금 생각해 주시는 게 여기서 이거 갖고서 그러면 이 로그립에 맞는 우리가 데이터를 만들어주자라고 했을 때 그러니까 지금 총 10개의 평가 기준이 평가 항목이 있고 3개의 스케일이 있으니까 이것만 놓고 보면 지금 그렇게 안 하고 있어.

참석자 1 01:41
그러니까 이제 이 부분이 아마 정의가 좀 안 된 느낌이어 가지고 지금은 이 부장님이 해 주시는 거는 레벨 별로 그러니까 베이식이라는 거에 대해서 한 그때 아마 30개 정도 수주해 주셨나요?
제가 20개 20개 근데 이 20개에 대한 매트릭을 이 부장님은 리카운트랑 무슨 램프랑 그리고

참석자 2 02:05
문단 수 수 아무튼 센텐스 카운트 어려운 단어 개수 AR 레벨 결국 AR 레벨 그게 이제 결과 값에 대한 리베이베이션이고 생성할 때는 베이직이랑 인터네이데이트 그거에 대한 상무님이 적어주신 파라미터 그거 반영해



## 2) Prompts — V0.0.1 (간단 요약) vs V0.0.2 (구조화 회의록)

- **V0.0.1**: 목적 중심의 간단 회의 요약 (한 문단 + bullet 5개 이하)  
- **V0.0.2**: 회사용 구조화 회의록 (Decisions / Action Items / Key Points / Risks / Open Questions / Next Steps)  


In [1]:

V001_SIMPLE_SUMMARY = '''You are a note-taker for a company meeting.
Summarize the meeting transcript concisely for busy stakeholders.
Output:
- 1-paragraph abstract (<= 120 words)
- Top-5 bullets of key points
Be neutral and avoid speculation. Do not invent facts. Use present tense.
'''

V002_STRUCTURED_MINUTES = '''You are an expert corporate minute-taker.
Extract structured minutes from the transcript with **no fabrication**.
Return Markdown with the following sections (use headings exactly):
# Decisions
- [Decision] <concise statement> (owner if applicable)

# Action Items
- [Task] <what> — Owner: <name or "TBD"> — Due: <YYYY-MM-DD or "TBD">

# Key Discussion Points
- <point 1>
- <point 2>

# Risks
- <risk 1 (cause → impact → mitigation)>

# Open Questions
- <question 1>

# Next Steps
- <step 1 (who/when)>

Rules:
- Quote numbers/dates only if present in the transcript; otherwise write "TBD".
- Do not include PII beyond names mentioned.
- Keep each bullet <= 25 words.
'''


In [2]:

# --- 3) LLM 호출 추상화 -----------------------------------------------------------
def call_openai_chat(system_prompt: str, user_text: str, model: str = "gpt-4o-mini", temperature: float = 0.2) -> str:
    """LLM 호출. OPENAI_API_KEY가 없으면 모의 출력(mock)으로 대체."""
    if os.getenv("OPENAI_API_KEY"):
        try:
            # Try official openai package first
            try:
                from openai import OpenAI
                client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
                resp = client.chat.completions.create(
                    model=model,
                    messages=[
                        {"role": "system", "content": system_prompt},
                        {"role": "user", "content": user_text},
                    ],
                    temperature=temperature,
                )
                return resp.choices[0].message.content
            except Exception:
                # Fallback to raw HTTP if new package not available
                import requests
                headers = {
                    "Authorization": f"Bearer {os.getenv('OPENAI_API_KEY')}",
                    "Content-Type": "application/json",
                }
                payload = {
                    "model": model,
                    "messages": [
                        {"role": "system", "content": system_prompt},
                        {"role": "user", "content": user_text},
                    ],
                    "temperature": temperature,
                }
                r = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload, timeout=60)
                r.raise_for_status()
                data = r.json()
                return data["choices"][0]["message"]["content"]
        except Exception as e:
            print("OpenAI call failed, falling back to mock. Error:", e)
    # Mock output for offline demo
    return f"""[MOCK OUTPUT]\nSystem: {system_prompt.splitlines()[0]}\nUserInputPreview: {user_text[:120]}...\n- Bullet 1\n- Bullet 2\n- Bullet 3"""


In [9]:

# --- 4) Langfuse 초기화 & Tracing 유틸 -------------------------------------------
USE_SDK = False
langfuse = None

try:
    from langfuse import get_client, observe
    from langfuse.openai import OpenAI as LFOpenAI  # optional
    langfuse = get_client()
    if langfuse and langfuse.auth_check():
        USE_SDK = True
        print("Langfuse SDK authenticated.")
    else:
        print("Langfuse SDK not authenticated; will use HTTP or no-op.")
except Exception as e:
    print("Langfuse SDK not available; continuing without it.", e)

def start_trace(name: str, metadata: Dict[str, Any] = None):
    if USE_SDK:
        # Start span as root trace context manager
        return langfuse.start_as_current_span(name=name, metadata=metadata or {})
    else:
        # No-op context manager
        from contextlib import contextmanager
        @contextmanager
        def _noop():
            class Dummy:
                def score_trace(self, *args, **kwargs): pass
                def update_trace(self, *args, **kwargs): pass
            yield Dummy()
        return _noop()

def log_trace_io(input_obj: Any, output_obj: Any):
    if USE_SDK:
        try:
            # update current trace input/output to enable dataset evals linkage
            langfuse.update_current_trace(input=input_obj, output=output_obj)
        except Exception as e:
            print("Failed to update current trace:", e)

def flush_langfuse():
    try:
        if USE_SDK:
            langfuse.flush()
    except Exception:
        pass


Langfuse SDK authenticated.


In [10]:

# --- 5) 두 가지 Prompt 실행 + Tracing --------------------------------------------
subset = transcript_text[:4000]  # 데모를 위해 앞부분만 사용

results = {}

with start_trace(name="meeting-minutes:v0.0.1", metadata={"version": "v0.0.1", "label": "dev", "use_case":"meeting-minutes"}) as span:
    out1 = call_openai_chat(V001_SIMPLE_SUMMARY, subset)
    results["v0.0.1"] = out1
    log_trace_io({"transcript": subset, "prompt_version": "v0.0.1"}, {"minutes": out1})

with start_trace(name="meeting-minutes:v0.0.2", metadata={"version": "v0.0.2", "label": "staging", "use_case":"meeting-minutes"}) as span:
    out2 = call_openai_chat(V002_STRUCTURED_MINUTES, subset)
    results["v0.0.2"] = out2
    log_trace_io({"transcript": subset, "prompt_version": "v0.0.2"}, {"minutes": out2})

flush_langfuse()

print("\n=== V0.0.1 (simple) ===\n", results["v0.0.1"][:800], "...")
print("\n=== V0.0.2 (structured) ===\n", results["v0.0.2"][:800], "...")



=== V0.0.1 (simple) ===
 **Abstract:**  
The meeting discusses the development of a service based on LLMOPS, focusing on the creation of evaluation metrics and data generation processes. Participants emphasize the need for clear definitions and parameters for evaluating generated content, including aspects such as sentence complexity and vocabulary difficulty. They explore the automation of data generation and the importance of diverse outputs for effective evaluation. The conversation highlights the necessity of refining the rubric and establishing a systematic approach to ensure that generated data meets the desired quality and variation.

**Key Points:**
- Discussion on LLMOPS-based service creation and evaluation metrics.
- Emphasis on defining parameters for content evaluation, including sentence complexity a ...

=== V0.0.2 (structured) ===
 # Decisions
- Define evaluation metrics for data generation (owner: TBD)

# Action Items
- Create a detailed rubric for evaluation metrics 


## 6) Langfuse Dataset 생성 & 아이템 업로드 (샘플 2개)

- 회의록 일부를 input으로, 기대 구조를 reference로 저장합니다.  
- 이후 Langfuse에서 **Dataset Run**을 실행하여, 각 Prompt 버전의 결과를 비교/평가할 수 있습니다.


### JSONL 파일에 저장된 내용

선택된 코드(7번 셀)에서 로컬 `datasets/week03_meeting_minutes_demo.jsonl` 파일에 **데이터셋 샘플 2개를 JSON Lines 형식으로 저장**합니다. JSONL은 각 줄이 하나의 JSON 객체인 텍스트 파일입니다.

#### 저장된 데이터 구조
파일 내용 예시 (실제 파일을 읽어보면):


In [ ]:
{"input": {"transcript": "회의 텍스트 일부..."}, "expected_output": {"sections": ["Decisions", "Action Items", "Key Discussion Points", "Next Steps"]}}
{"input": {"transcript": "회의 텍스트 다른 일부..."}, "expected_output": {"sections": ["Decisions", "Action Items", "Risks", "Open Questions", "Next Steps"]}}



- **각 줄**: `item1`과 `item2` 딕셔너리를 `json.dumps()`로 JSON 문자열로 변환.
- **item1**: 회의 텍스트 앞부분 + 기대 섹션 4개.
- **item2**: 회의 텍스트 중간부분 + 기대 섹션 5개.
- **목적**: Langfuse SDK 없이 수동으로 데이터셋 업로드할 때 백업용.

#### 확인 방법
터미널에서 `cat datasets/week03_meeting_minutes_demo.jsonl` 실행하면 내용 확인 가능. 

In [11]:

# Dataset 이름
DATASET_NAME = "week03_meeting_minutes_demo"

# 샘플 아이템 2개 (간단 예시)
item1 = {
    "input": {"transcript": subset[:1500]},
    "expected_output": {
        "sections": ["Decisions","Action Items","Key Discussion Points","Next Steps"],  # 기대 섹션 존재성
    }
}
item2 = {
    "input": {"transcript": subset[1500:3000] if len(subset) > 3000 else subset},
    "expected_output": {
        "sections": ["Decisions","Action Items","Risks","Open Questions","Next Steps"],
    }
}

created = False
if USE_SDK:
    try:
        langfuse.create_dataset(name=DATASET_NAME)
        for item in (item1, item2):
            langfuse.create_dataset_item(
                dataset_name=DATASET_NAME,
                input=item["input"],
                expected_output=item["expected_output"]
            )
        created = True
        print(f"Created dataset '{DATASET_NAME}' with 2 items via SDK.")
    except Exception as e:
        print("SDK dataset creation failed:", e)

# 로컬 JSONL도 함께 저장(백업/수동 업로드용)
ds_dir = Path("datasets"); ds_dir.mkdir(exist_ok=True)
jsonl_path = ds_dir / f"{DATASET_NAME}.jsonl"
with open(jsonl_path, "w", encoding="utf-8") as f:
    for item in (item1, item2):
        f.write(json.dumps(item, ensure_ascii=False) + "\n")
print("Local backup dataset written to", jsonl_path.resolve())


Created dataset 'week03_meeting_minutes_demo' with 2 items via SDK.
Local backup dataset written to /home/dhc99/ajou-llmops-2025-2nd-semester/week03/datasets/week03_meeting_minutes_demo.jsonl



## 7) (선택) Native Dataset Run 실행 예시

아래 코드는 Langfuse **Datasets Cookbook**의 패턴을 따라, 각 Dataset Item을 순회하며 애플리케이션을 실행하고  
`root_span.score_trace(...)` 등으로 평가 스코어를 기록합니다.  
실행 전, SDK 인증이 되어 있어야 합니다.



### 선택된 코드 설명 (8번 셀: Native Dataset Run 실행 예시)

이 코드는 **Langfuse 데이터셋을 기반으로 프롬프트 버전을 자동 평가**하는 역할을 합니다. 데이터셋의 각 샘플에 대해 LLM을 실행하고, 결과를 트레이싱하며 성능 지표를 계산합니다.

#### 주요 함수
1. **`presence_of_sections_eval(output_text, expected_sections)`**:
   - **목적**: LLM 출력에 기대 섹션(예: "# Decisions", "# Action Items")이 얼마나 포함되는지 확인.
   - **로직**: 텍스트에서 섹션 헤더를 검색해 일치 개수를 세고, 총 기대 섹션 수로 나누어 0.0~1.0 스코어 반환.
   - **예시**: 기대 섹션 4개 중 3개 포함 → 0.75.

2. **`run_dataset_experiment(run_name, system_prompt)`**:
   - **목적**: 데이터셋 아이템을 순회하며 프롬프트 실행 → 트레이싱 → 평가.
   - **로직**:
     - 데이터셋 로드 (`langfuse.get_dataset(DATASET_NAME)`).
     - 각 아이템에 대해: LLM 호출 → 입력/출력 트레이스 기록 → 섹션 커버리지 스코어 계산 → Langfuse에 기록.
     - `root_span.score_trace("section_coverage", score)`로 평가 지표 저장.
   - **출력**: "Finished dataset run 'minutes_v0.0.2' on dataset 'week03_meeting_minutes_demo'."

#### 왜 필요하나?
- **자동 평가**: 수동 비교 대신 데이터셋 기반으로 프롬프트 성능 측정 (예: 구조화 프롬프트가 섹션을 더 잘 생성하는지).
- **Langfuse 연동**: 트레이스와 스코어를 콘솔에서 확인 가능.
- **실행**: 주석 해제 후 `run_dataset_experiment(run_name="minutes_v0.0.2", system_prompt=V002_STRUCTURED_MINUTES)` 호출.

이 셀은 프롬프트 버전 비교의 핵심으로, 실행 후 Langfuse에서 "section_coverage" 지표를 분석할 수 있습니다. 

### 기대 섹션(Expected Sections)이란?

**기대 섹션**은 **LLM이 출력해야 하는 이상적인 구조(섹션 헤더)를 미리 정의한 리스트**입니다. 평가 시 LLM 출력에 이 섹션들이 얼마나 포함되는지 확인합니다.

#### 코드 예시


In [ ]:
item1 = {
    "input": {"transcript": subset[:1500]},
    "expected_output": {
        "sections": ["Decisions", "Action Items", "Key Discussion Points", "Next Steps"],  # ← 기대 섹션
    }
}



- **출처**: 데이터셋 생성 시 수동 정의 (V002_STRUCTURED_MINUTES 프롬프트 기반).
- **목적**: 프롬프트의 구조화 성능 평가 기준.

#### 예시
- **프롬프트 V0.0.2**: "# Decisions", "# Action Items" 등 섹션 생성 지시.
- **기대 섹션**: `["Decisions", "Action Items", "Risks", "Open Questions", "Next Steps"]`
- **평가**: 출력에 이 섹션들이 포함되면 Score 상승.

기대 섹션은 "이 프롬프트로 이 구조를 만들어야 함"을 명시한 기준입니다. 더 궁금한 점 있으신가요?

In [12]:

def presence_of_sections_eval(output_text: str, expected_sections) -> float:
    """출력에 예상 섹션 헤더가 얼마나 포함되는지 0.0~1.0 반환."""
    if not output_text:
        return 0.0
    hits = 0
    for sec in expected_sections:
        if f"# {sec}" in output_text or sec in output_text:
            hits += 1
    return hits / max(1, len(expected_sections))

def run_dataset_experiment(run_name: str, system_prompt: str):
    if not USE_SDK:
        print("Langfuse SDK unavailable; skipping remote run.")
        return
    dataset = langfuse.get_dataset(DATASET_NAME)
    for item in dataset.items:
        with item.run(run_name=run_name) as root_span:
            output = call_openai_chat(system_prompt, item.input["transcript"])
            # Trace IO를 업데이트해야 Langfuse의 Eval 기능에서 인풋/아웃풋을 인식합니다.
            root_span.update_trace(input=item.input, output=output)
            # 간단한 구조성 스코어 예시
            score = presence_of_sections_eval(output, item.expected_output.get("sections", []))
            root_span.score_trace(name="section_coverage", value=score)
    langfuse.flush()
    print(f"Finished dataset run '{run_name}' on dataset '{DATASET_NAME}'.")

# 예시 실행 (주석 해제하여 사용)
run_dataset_experiment(run_name="minutes_v0.0.2", system_prompt=V002_STRUCTURED_MINUTES)


Finished dataset run 'minutes_v0.0.2' on dataset 'week03_meeting_minutes_demo'.



## 8) (선택) Langfuse Prompt 버전 생성 & 라벨 지정 (REST API 예시)

- 동일 `promptName`으로 새 버전을 생성하면 자동으로 버전이 증분됩니다.  
- `labels`에 `staging`, `production` 등을 부여/변경하여 배포 포인터로 사용합니다.


### 이 JSON 객체 설명

이건 **Langfuse API에서 프롬프트 버전을 생성한 후 반환된 응답 데이터**입니다. `create_prompt_version_via_rest` 함수 실행 결과예요.

#### 주요 필드 설명
- **`id`**: 프롬프트 버전의 고유 ID (예: `29d29769-8408-48ca-907a-3238fef4b52e`).
- **`createdAt` / `updatedAt`**: 생성/수정 시간 (ISO 8601 형식, UTC).
- **`projectId`**: Langfuse 프로젝트 ID.
- **`createdBy`**: 생성자 ("API"로 API 호출).
- **`prompt`**: 업로드된 프롬프트 텍스트 (V002_STRUCTURED_MINUTES 내용).
- **`name`**: 프롬프트 이름 ("meeting-minutes").
- **`version`**: 버전 번호 (2, 자동 증분).
- **`type`**: 프롬프트 타입 ("text").
- **`config`**: 모델 설정 (model_name: "gpt-4o-mini", temperature: 0.2).
- **`labels`**: 태그 리스트 (["production", "latest"] – 배포 환경 지정).
- **`tags`**: 추가 태그 (빈 리스트).

#### 왜 반환되나?
- **성공 확인**: 함수가 정상 실행되면 이 JSON을 반환 (실패 시 에러).
- **메타데이터**: Langfuse 콘솔에서 이 버전을 조회/관리할 때 사용.

이 데이터는 프롬프트 버전이 성공적으로 생성되었음을 의미합니다. Langfuse 콘솔에서 확인 가능해요. 더 궁금한 점 있으신가요?

In [13]:

import base64, json

def lf_auth_headers():
    token = base64.b64encode(f"{LANGFUSE_PUBLIC_KEY}:{LANGFUSE_SECRET_KEY}".encode()).decode()
    return {"Authorization": f"Basic {token}", "Content-Type": "application/json"}

def create_prompt_version_via_rest(prompt_name: str, prompt_text: str, model_name: str, labels=None):
    if not (LANGFUSE_PUBLIC_KEY and LANGFUSE_SECRET_KEY):
        print("Missing Langfuse credentials.")
        return None
    import requests
    url = f"{LANGFUSE_HOST.rstrip('/')}/api/public/v2/prompts"
    payload = {
        "name": prompt_name,
        "prompt": prompt_text,
        "config": {"model_name": model_name, "temperature": 0.2},
        "labels": labels or []
    }
    r = requests.post(url, headers=lf_auth_headers(), json=payload, timeout=30)
    try:
        r.raise_for_status()
    except Exception as e:
        print("Prompt creation failed:", r.status_code, r.text)
        raise
    return r.json()

# 예시: v0.1 → v0.2 (staging → production) 업로드 (주석 해제 후 사용)
create_prompt_version_via_rest("meeting-minutes", V001_SIMPLE_SUMMARY, "gpt-4o-mini", labels=["staging"])
create_prompt_version_via_rest("meeting-minutes", V002_STRUCTURED_MINUTES, "gpt-4o-mini", labels=["production"])


{'id': '29d29769-8408-48ca-907a-3238fef4b52e',
 'createdAt': '2025-10-06T09:02:29.239Z',
 'updatedAt': '2025-10-06T09:02:29.239Z',
 'projectId': 'cmgdwile006tdad0774oq3lk9',
 'createdBy': 'API',
 'prompt': 'You are an expert corporate minute-taker.\nExtract structured minutes from the transcript with **no fabrication**.\nReturn Markdown with the following sections (use headings exactly):\n# Decisions\n- [Decision] <concise statement> (owner if applicable)\n\n# Action Items\n- [Task] <what> — Owner: <name or "TBD"> — Due: <YYYY-MM-DD or "TBD">\n\n# Key Discussion Points\n- <point 1>\n- <point 2>\n\n# Risks\n- <risk 1 (cause → impact → mitigation)>\n\n# Open Questions\n- <question 1>\n\n# Next Steps\n- <step 1 (who/when)>\n\nRules:\n- Quote numbers/dates only if present in the transcript; otherwise write "TBD".\n- Do not include PII beyond names mentioned.\n- Keep each bullet <= 25 words.\n',
 'name': 'meeting-minutes',
 'version': 2,
 'type': 'text',
 'isActive': None,
 'config': {'mode


## 9) Prompty 파일 생성 (GitHub PR용)

- `prompts/meeting_minutes_v0.1.prompty`  
- `prompts/meeting_minutes_v0.2.prompty`  
를 생성합니다. (PR 시 `v0.1 → v0.2` 변경 경험 포함)


In [15]:
# --- 9) Prompty 파일 생성 (GitHub PR용) ---------------------------------------

from pathlib import Path

# Prompty 파일들을 위한 디렉토리 생성
prompts_dir = Path("week03/prompts")
prompts_dir.mkdir(parents=True, exist_ok=True)

# V0.1 Prompty 파일 생성
v01_prompty = """name: Meeting Minutes - Simple Summary
version: v0.1.0
description: Summarize a meeting transcript into a short abstract and top-5 bullets.
inputs:
  transcript:
    type: string
outputs:
  response_format: markdown
---
system:
You are a note-taker for a company meeting.
Summarize the meeting transcript concisely for busy stakeholders.
Output:
- 1-paragraph abstract (<= 120 words)
- Top-5 bullets of key points
Be neutral and avoid speculation. Do not invent facts. Use present tense.
user:
{{ transcript }}
"""

# V0.2 Prompty 파일 생성
v02_prompty = """name: Meeting Minutes - Structured Corporate
version: v0.2.0
description: Produce structured company-grade minutes with Decisions, Action Items (owner/due), Risks, etc.
inputs:
  transcript:
    type: string
outputs:
  response_format: markdown
---
system:
You are an expert corporate minute-taker.
Extract structured minutes from the transcript with **no fabrication**.
Return Markdown with the following sections (use headings exactly):
# Decisions
- [Decision] <concise statement> (owner if applicable)

# Action Items
- [Task] <what> — Owner: <name or "TBD"> — Due: <YYYY-MM-DD or "TBD">

# Key Discussion Points
- <point 1>
- <point 2>

# Risks
- <risk 1 (cause → impact → mitigation)>

# Open Questions
- <question 1>

# Next Steps
- <step 1 (who/when)>

Rules:
- Quote numbers/dates only if present in the transcript; otherwise write "TBD".
- Do not include PII beyond names mentioned.
- Keep each bullet <= 25 words.
user:
{{ transcript }}
"""

# 파일 저장
with open(prompts_dir / "meeting_minutes_v0.1.prompty", "w", encoding="utf-8") as f:
    f.write(v01_prompty)

with open(prompts_dir / "meeting_minutes_v0.2.prompty", "w", encoding="utf-8") as f:
    f.write(v02_prompty)

print("Created Prompty files:")
print("- meeting_minutes_v0.1.prompty")
print("- meeting_minutes_v0.2.prompty")

Created Prompty files:
- meeting_minutes_v0.1.prompty
- meeting_minutes_v0.2.prompty


In [17]:

from pathlib import Path
print("Prompty files:")
for p in Path("week03/prompts").glob("*.prompty"):
    print("-", p.name)


Prompty files:
- meeting_minutes_v0.1.prompty
- meeting_minutes_v0.2.prompty



## 10) 마무리 & 다음 단계
1. **Tracing 확인**: Langfuse 콘솔에서 오늘 생성된 트레이스를 확인합니다.  
2. **Dataset Run**: `run_dataset_experiment(...)` 실행 후 지표(예: `section_coverage`) 확인.  
3. **Prompt 배포 라벨**: REST 또는 UI로 `staging` → `production` 라벨 전환.  
4. **GitHub PR**: `prompts/meeting_minutes_v0.1.prompty` → `v0.2` 변경 포함 PR 생성.  
5. **초대**: 프로젝트에 `smilechacha@ajou.ac.kr` 뷰어 이상 권한 초대.
